# Kaggle Categorical competition

In [191]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn import metrics
import xgboost
import lightgbm
import catboost
from catboost import CatBoostClassifier, CatBoostRegressor
from warnings import filterwarnings
import pickle
filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')
df_sample=pd.read_csv('sample_submission.csv')

In [3]:
df_sample.head()

,id,target
0,600000,0.5
1,600001,0.5
2,600002,0.5
3,600003,0.5
4,600004,0.5


In [4]:
df_train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,...,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,...,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,...,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,...,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,...,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0


In [5]:
df_test.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,600000,0.0,0.0,0.0,F,Y,Blue,Polygon,Axolotl,Finland,...,ca9ad1d4b,fced9e114,3.0,Novice,Boiling Hot,f,U,oU,3.0,9.0
1,600001,0.0,0.0,0.0,F,Y,Red,Circle,Lion,Russia,...,060a21580,7ca8775da,1.0,Novice,Cold,n,N,NaN,2.0,8.0
2,600002,0.0,0.0,0.0,F,Y,Blue,Circle,Axolotl,Russia,...,165e81a00,5940334c9,1.0,Expert,Warm,i,N,DN,2.0,6.0
3,600003,1.0,0.0,0.0,F,N,Red,Polygon,Axolotl,Costa Rica,...,77d41330d,6fbdeefc8,1.0,Expert,Hot,m,B,AG,1.0,6.0
4,600004,0.0,0.0,1.0,F,Y,Red,Circle,NaN,Finland,...,2218d9dfe,2a27c8fde,1.0,Contributor,Lava Hot,o,J,DT,3.0,3.0


In [6]:
df_train.isna().any()

id        False
bin_0      True
bin_1      True
bin_2      True
bin_3      True
bin_4      True
nom_0      True
nom_1      True
nom_2      True
nom_3      True
nom_4      True
nom_5      True
nom_6      True
nom_7      True
nom_8      True
nom_9      True
ord_0      True
ord_1      True
ord_2      True
ord_3      True
ord_4      True
ord_5      True
day        True
month      True
target    False
dtype: bool

In [7]:
df_train.isna().sum()

id            0
bin_0     17894
bin_1     18003
bin_2     17930
bin_3     18014
bin_4     18047
nom_0     18252
nom_1     18156
nom_2     18035
nom_3     18121
nom_4     18035
nom_5     17778
nom_6     18131
nom_7     18003
nom_8     17755
nom_9     18073
ord_0     18288
ord_1     18041
ord_2     18075
ord_3     17916
ord_4     17930
ord_5     17713
day       17952
month     17988
target        0
dtype: int64

In [8]:
df_train['bin_0'].unique()

array([ 0.,  1., nan])

In [9]:
df_train['nom_1'].unique()

array(['Trapezoid', 'Star', nan, 'Circle', 'Triangle', 'Polygon',
       'Square'], dtype=object)

In [10]:
for col in df_train.columns:
    print('column {} ----- {} unique values ---- type: {} '.format(col, df_train[col].nunique(), df_train[col].dtype.name))


column id ----- 600000 unique values ---- type: int64 
column bin_0 ----- 2 unique values ---- type: float64 
column bin_1 ----- 2 unique values ---- type: float64 
column bin_2 ----- 2 unique values ---- type: float64 
column bin_3 ----- 2 unique values ---- type: object 
column bin_4 ----- 2 unique values ---- type: object 
column nom_0 ----- 3 unique values ---- type: object 
column nom_1 ----- 6 unique values ---- type: object 
column nom_2 ----- 6 unique values ---- type: object 
column nom_3 ----- 6 unique values ---- type: object 
column nom_4 ----- 4 unique values ---- type: object 
column nom_5 ----- 1220 unique values ---- type: object 
column nom_6 ----- 1519 unique values ---- type: object 
column nom_7 ----- 222 unique values ---- type: object 
column nom_8 ----- 222 unique values ---- type: object 
column nom_9 ----- 2218 unique values ---- type: object 
column ord_0 ----- 3 unique values ---- type: float64 
column ord_1 ----- 5 unique values ---- type: object 
column ord

### Applying labelencoding to all columns

In [10]:
# le = LabelEncoder()
# df_train['bin_0'] = le.fit_transform(df_train['bin_0'])
# df_train['bin_0'].head()

In [11]:
# df_train['bin_0'].nunique()

In [12]:
# df_train['bin_3'].unique()

In [13]:
# df_train['bin_3'].head()

### A lable encoder needs to keep null values while encoding everything else. This **[link](https://stackoverflow.com/questions/54444260/labelencoder-that-keeps-missing-values-as-nan)** explains how it can be implemented.

In [11]:
df_train_cleaned = df_train.drop(['id', 'target'], axis = 1)
df_train_cleaned.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,...,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0
1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,...,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0
2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,Bassoon,...,745b909d1,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0
3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,...,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0
4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,...,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0


In [12]:
df_test.drop('id', axis = 1, inplace=True)
# df_test.head()

In [13]:
encoders = dict()

for col_name in df_train_cleaned.columns:
    if df_train_cleaned[col_name].dtype == 'object':
        label_encoder = LabelEncoder()
        series = df_train_cleaned[col_name]
        df_train_cleaned[col_name] = pd.Series(
            label_encoder.fit_transform(series[series.notnull()]),
            index=series[series.notnull()].index
        )
        
        # This will cause the following error due to seeing unseen values for the first time in test data frame
        # ValueError: y contains previously unseen labels: 'a885aacec'
        series = df_test[col_name]
        df_test[col_name] = pd.Series(
            label_encoder.transform(series[series.notnull()]),
            index=series[series.notnull()].index
        )

        encoders[col_name] = label_encoder

ValueError: y contains previously unseen labels: 'a885aacec'

## Preveting above error and keeping null values 
To prevent getting ValueError: y contains previously unseen labels: 'a885aacec'
+ concat both dataframes and apply encoder simultaneously
+ df_le will be used for encoding values with two conditions:
 1. Null values will stay null.
 2. Transforming values with encoder will not fail due to seeing values for the first time.
 
After the transform is done, split both dataframes accordingly to have train and test result.

In [14]:
df_le = pd.concat([df_train_cleaned, df_test], ignore_index=True)
df_le.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,3.0,5.0,0.0,...,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0
1,1.0,1.0,0.0,0.0,1.0,2.0,3.0,0.0,NaN,3.0,...,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0
2,0.0,1.0,0.0,0.0,0.0,2.0,NaN,3.0,0.0,0.0,...,745b909d1,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0
3,NaN,0.0,0.0,0.0,0.0,2.0,0.0,3.0,3.0,3.0,...,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0
4,0.0,NaN,0.0,1.0,0.0,2.0,5.0,3.0,2.0,NaN,...,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0


In [18]:
encoders = dict()

for col_name in df_le.columns:
    if df_le[col_name].dtype == 'object':
        label_encoder = LabelEncoder()
        series = df_le[col_name].apply(str)
        df_le[col_name] = pd.Series(
            label_encoder.fit_transform(series[series.notnull()]),
            index=series[series.notnull()].index
        )

        encoders[col_name] = label_encoder

In [52]:
encoders

{'nom_6': LabelEncoder(),
 'nom_7': LabelEncoder(),
 'nom_8': LabelEncoder(),
 'nom_9': LabelEncoder(),
 'ord_1': LabelEncoder(),
 'ord_2': LabelEncoder(),
 'ord_3': LabelEncoder(),
 'ord_4': LabelEncoder(),
 'ord_5': LabelEncoder()}

In [19]:
# df_train.head()
df_le.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,3.0,5.0,0.0,...,1,27,3.0,0,3,2,20,56,6.0,3.0
1,1.0,1.0,0.0,0.0,1.0,2.0,3.0,0.0,NaN,3.0,...,69,2112,3.0,2,5,4,23,151,7.0,7.0
2,0.0,1.0,0.0,0.0,0.0,2.0,NaN,3.0,0.0,0.0,...,102,2218,3.0,5,2,13,15,105,5.0,9.0
3,NaN,0.0,0.0,0.0,0.0,2.0,0.0,3.0,3.0,3.0,...,170,2167,1.0,4,4,0,2,140,3.0,3.0
4,0.0,NaN,0.0,1.0,0.0,2.0,5.0,3.0,2.0,NaN,...,222,1747,3.0,2,1,7,2,50,5.0,12.0


In [22]:
df_train.index

RangeIndex(start=0, stop=600000, step=1)

In [23]:
len(df_train)

600000

In [24]:
len(df_test)

400000

In [25]:
len(df_le)

1000000

In [26]:
df_train_drp = df_le.iloc[:600000]

In [28]:
df_test_drp = df_le.iloc[600000:]

In [27]:
df_train_drp.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,3.0,5.0,0.0,...,1,27,3.0,0,3,2,20,56,6.0,3.0
1,1.0,1.0,0.0,0.0,1.0,2.0,3.0,0.0,NaN,3.0,...,69,2112,3.0,2,5,4,23,151,7.0,7.0
2,0.0,1.0,0.0,0.0,0.0,2.0,NaN,3.0,0.0,0.0,...,102,2218,3.0,5,2,13,15,105,5.0,9.0
3,NaN,0.0,0.0,0.0,0.0,2.0,0.0,3.0,3.0,3.0,...,170,2167,1.0,4,4,0,2,140,3.0,3.0
4,0.0,NaN,0.0,1.0,0.0,2.0,5.0,3.0,2.0,NaN,...,222,1747,3.0,2,1,7,2,50,5.0,12.0


In [29]:
len(df_train_drp)

600000

In [30]:
df_test_drp.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
600000,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,2.0,...,174,2196,3.0,4,0,5,20,147,3.0,9.0
600001,0.0,0.0,0.0,0.0,1.0,2.0,0.0,4.0,5.0,0.0,...,4,1107,1.0,4,1,13,13,140,2.0,8.0
600002,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,3.0,...,16,812,1.0,1,5,8,13,12,2.0,6.0
600003,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,0.0,...,107,996,1.0,1,3,12,1,0,1.0,6.0
600004,0.0,0.0,1.0,0.0,1.0,2.0,0.0,NaN,3.0,3.0,...,30,371,1.0,0,4,15,9,14,3.0,3.0


In [31]:
len(df_test_drp)

400000

### Applying an xgboost to the dataframe

In [32]:
# X = df_train.drop(['id', 'target'], axis=1)
# y = df_train['target']

X = df_train_drp
y = df_train['target']

In [33]:
test_size = 0.3
train_size = 0.7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, train_size=train_size, random_state=2021)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

xgb = xgboost.XGBRegressor(n_estimator=20000, max_depth=8, colsample_bytree=0.85,
                           reg_alpha=0.05, reg_lambda=0.05, random_state=2021,learning_rate=0.008, verbosity=1)

In [34]:
xgb.fit(X_train, y_train, eval_set=[(X_val,y_val)], early_stopping_rounds=50, verbose=50 )

[22:54:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:0.49823
[50]	validation_0-rmse:0.43396
[99]	validation_0-rmse:0.40130


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.85, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.008, max_delta_step=0, max_depth=8,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimator=20000, n_estimators=100, n_jobs=8, num_parallel_tree=1,
             random_state=2021, reg_alpha=0.05, reg_lambda=0.05,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=1)

#### Accuracy is not the right metric for this model. 

[Why accuracy is not the right metric for regression models is explained here.](https://stackoverflow.com/questions/38015181/accuracy-score-valueerror-cant-handle-mix-of-binary-and-continuous-target/54458777) 

trying to use a classification metric (accuracy) in a regression (i.e. numeric prediction) model (LinearRegression), which is meaningless

In [35]:
xgb.score(X_test, y_test)

-0.05590674376838023

In [36]:
xgb.predict(X_test)

array([0.26129925, 0.3036174 , 0.28389853, ..., 0.36175764, 0.29325464,
       0.3128003 ], dtype=float32)

In [37]:
y_test.unique()

array([0, 1], dtype=int64)

#### The following is a workaround to calculate accuracy_score, but it is completely wrong. This metric is for classification and not regression.

In [38]:
metrics.accuracy_score(y_test, xgb.predict(X_test).round())

0.8133777777777778

### Model score

A set of right type of metrics for regression is available [here](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

In [39]:
pred = xgb.predict(X_test)
metrics.r2_score(y_test, pred)

-0.05590674376838023

In [40]:
metrics.explained_variance_score(y_test, pred)

0.07149705805443318

In [41]:
metrics.max_error(y_test, pred)

0.7638150006532669

In [42]:
metrics.mean_absolute_error(y_test, pred)

0.3782907192962037

In [43]:
metrics.mean_squared_error(y_test, pred)

0.16106600599479723

In [44]:
metrics.mean_poisson_deviance(y_test, pred)

0.6629741217403212

In [45]:
# metrics.mean_gamma_deviance(y_test, pred)
# ValueError: Mean Tweedie deviance error with power=2 can only be used on strictly positive y and y_pred.

In [46]:
metrics.mean_tweedie_deviance(y_test, pred)

0.16106600599479723

## Changing encoding

+ nom_5: count encoding
+ nom_6: count encoding
+ nom_7: count encoding
+ nom_8: count encoding
+ nom_9: count encoding

To apply different encoding types, make a method to apply different encodings:

### Making a custome encoding method

In [86]:
def apply_custom_encoder(encoder_type, dftrain, dftest):
    
    train_idx = len(df_train)
    df_tmp = pd.concat([dftrain, dftest], ignore_index=True)
    encoders_df = dict()

    for col_name in df_le.columns:
        if df_tmp[col_name].dtype == 'object':
            encoder = encoder_type()
            series = df_tmp[col_name].apply(str)
            df_tmp[col_name] = pd.Series(
                encoder.fit_transform(series[series.notnull()]),
                index=series[series.notnull()].index
            )

            encoders_df[col_name] = encoder
    
    dftrain= df_tmp.iloc[:train_idx]
    dftest = df_tmp.iloc[train_idx:]
    return (dftrain, dftest, encoders_df)

In [106]:
def apply_custom_encoder_cols(encoder_type, dftrain, dftest, cols=None):
    
    train_idx = len(df_train)
    df_tmp = pd.concat([dftrain, dftest], ignore_index=True)
    encoders_df = dict()

    for col_name in cols:
        if df_tmp[col_name].dtype == 'object':
            encoder = encoder_type()
            series = df_tmp[col_name].apply(str)
            df_tmp[col_name] = pd.Series(
                encoder.fit_transform(series[series.notnull()]),
                index=series[series.notnull()].index
            )

            encoders_df[col_name] = encoder
    
    dftrain= df_tmp.iloc[:train_idx]
    dftest = df_tmp.iloc[train_idx:]
    return (dftrain, dftest, encoders_df)

In [107]:
df1, df2, _ = apply_custom_encoder(LabelEncoder, df_train_drp, df_test_drp)

In [108]:
df1.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,3.0,5.0,0.0,...,1,27,3.0,0,3,2,20,56,6.0,3.0
1,1.0,1.0,0.0,0.0,1.0,2.0,3.0,0.0,NaN,3.0,...,69,2112,3.0,2,5,4,23,151,7.0,7.0
2,0.0,1.0,0.0,0.0,0.0,2.0,NaN,3.0,0.0,0.0,...,102,2218,3.0,5,2,13,15,105,5.0,9.0
3,NaN,0.0,0.0,0.0,0.0,2.0,0.0,3.0,3.0,3.0,...,170,2167,1.0,4,4,0,2,140,3.0,3.0
4,0.0,NaN,0.0,1.0,0.0,2.0,5.0,3.0,2.0,NaN,...,222,1747,3.0,2,1,7,2,50,5.0,12.0


In [109]:
df1, df2, _ = apply_custom_encoder_cols(LabelEncoder, df_train_drp, df_test_drp, cols = ['nom_3', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9'])

In [110]:
df1.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,3.0,5.0,0.0,...,1,27,3.0,0,3,2,20,56,6.0,3.0
1,1.0,1.0,0.0,0.0,1.0,2.0,3.0,0.0,NaN,3.0,...,69,2112,3.0,2,5,4,23,151,7.0,7.0
2,0.0,1.0,0.0,0.0,0.0,2.0,NaN,3.0,0.0,0.0,...,102,2218,3.0,5,2,13,15,105,5.0,9.0
3,NaN,0.0,0.0,0.0,0.0,2.0,0.0,3.0,3.0,3.0,...,170,2167,1.0,4,4,0,2,140,3.0,3.0
4,0.0,NaN,0.0,1.0,0.0,2.0,5.0,3.0,2.0,NaN,...,222,1747,3.0,2,1,7,2,50,5.0,12.0


### Count encoding

Nominal features are replaced by their counts. A sample will be as follows: 

In [89]:
df_train_2 = df_train.copy()

In [90]:
# !pip install category_encoders

In [91]:
import category_encoders as ce

In [131]:
df_train_ce = pd.read_csv('Train.csv')
df_test_ce = pd.read_csv('Test.csv')
df_train_ce.drop(['id', 'target'], axis=1, inplace=True)
df_test_ce.drop('id', axis=1, inplace=True)
df_tmp = pd.concat([df_train_ce, df_test_ce], ignore_index=True)

In [132]:
cat_features = ['nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
count_enc = ce.CountEncoder()
count_encoded = count_enc.fit_transform(df_tmp[cat_features])

In [133]:
count_encoded.head()

,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9
0,199463,274196,199933,328104,653,210,3037,2684,323
1,23678,254155,30297,514158,1146,706,8161,4833,879
2,30103,274196,44044,328104,1205,1176,5486,7160,30133
3,175071,274196,174188,514158,1053,1056,5235,3696,251
4,273832,274196,253274,30028,1401,443,7483,29711,898


In [165]:
df_train_ce = pd.read_csv('Train.csv')
y = df_train_ce['target']
df_test_ce = pd.read_csv('Test.csv')
df_train_ce.drop(['id', 'target'], axis=1, inplace=True)
df_test_ce.drop('id', axis=1, inplace=True)
df_tmp = pd.concat([df_train_ce, df_test_ce], ignore_index=True)

cat_features = ['nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
count_enc = ce.CountEncoder(handle_missing='return_nan')
df_count = count_enc.fit_transform(df_tmp[cat_features])


In [166]:
# df_train_ce = pd.read_csv('Train.csv')
# df_test_ce = pd.read_csv('Test.csv')
# df_train_ce.drop(['target'], axis=1, inplace=True)
# df_tmp = pd.concat([df_train_ce, df_test_ce], ignore_index=True)

# cat_features = ['nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
# count_enc = ce.CountEncoder(handle_missing='return_nan')
# df_count = count_enc.fit_transform(df_tmp[cat_features])

In [167]:
df_count.head()

,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9
0,199463.0,274196.0,199933.0,328104.0,653.0,210.0,3037.0,2684.0,323.0
1,23678.0,254155.0,NaN,514158.0,1146.0,706.0,8161.0,4833.0,879.0
2,NaN,274196.0,44044.0,328104.0,1205.0,1176.0,5486.0,7160.0,NaN
3,175071.0,274196.0,174188.0,514158.0,1053.0,1056.0,5235.0,3696.0,251.0
4,273832.0,274196.0,253274.0,NaN,1401.0,443.0,7483.0,NaN,898.0


In [168]:
len(df_count)

1000000

In [169]:
# concatinating two dataframes column-wise
df_tmp2 = df_tmp.drop(cat_features, axis=1)
df_ce = pd.concat([df_tmp2, df_count], axis=1)
df_ce.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,ord_0,ord_1,ord_2,ord_3,...,month,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9
0,0.0,0.0,0.0,F,N,Red,3.0,Contributor,Hot,c,...,3.0,199463.0,274196.0,199933.0,328104.0,653.0,210.0,3037.0,2684.0,323.0
1,1.0,1.0,0.0,F,Y,Red,3.0,Grandmaster,Warm,e,...,7.0,23678.0,254155.0,NaN,514158.0,1146.0,706.0,8161.0,4833.0,879.0
2,0.0,1.0,0.0,F,N,Red,3.0,NaN,Freezing,n,...,9.0,NaN,274196.0,44044.0,328104.0,1205.0,1176.0,5486.0,7160.0,NaN
3,NaN,0.0,0.0,F,N,Red,1.0,Novice,Lava Hot,a,...,3.0,175071.0,274196.0,174188.0,514158.0,1053.0,1056.0,5235.0,3696.0,251.0
4,0.0,NaN,0.0,T,N,Red,3.0,Grandmaster,Cold,h,...,12.0,273832.0,274196.0,253274.0,NaN,1401.0,443.0,7483.0,NaN,898.0


In [160]:
# cat_features = ['nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
# # for col in cat_features:
# #     count_enc = ce.CountEncoder()
# #     df_train_2[col] = count_enc.fit_transform(df_train_2[col])


# df_train_ce = pd.read_csv('Train.csv')
# df_test_ce = pd.read_csv('Test.csv')
# df_train_ce.drop(['id', 'target'], axis=1, inplace=True)
# df_test_ce.drop('id', axis=1, inplace=True)
# df_tmp = pd.concat([df_train_ce, df_test_ce], ignore_index=True)

# encoders_ce = dict()

# for col_name in cat_features:
#     if df_tmp[col_name].dtype == 'object':
#         count_encoder = ce.CountEncoder()
#         series = df_tmp[col_name].apply(str)
        
#         #ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
#         df_tmp[col_name] = pd.Series(                   
#             count_encoder.fit_transform(series[series.notnull()]),
#             index=series[series.notnull()].index
#         )

#         encoders_ce[col_name] = count_encoder

In [170]:
df_train_ce = df_ce.iloc[:600000]
df_test_ce = df_ce.iloc[600000:]

In [171]:
df_train_ce.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,ord_0,ord_1,ord_2,ord_3,...,month,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9
0,0.0,0.0,0.0,F,N,Red,3.0,Contributor,Hot,c,...,3.0,199463.0,274196.0,199933.0,328104.0,653.0,210.0,3037.0,2684.0,323.0
1,1.0,1.0,0.0,F,Y,Red,3.0,Grandmaster,Warm,e,...,7.0,23678.0,254155.0,NaN,514158.0,1146.0,706.0,8161.0,4833.0,879.0
2,0.0,1.0,0.0,F,N,Red,3.0,NaN,Freezing,n,...,9.0,NaN,274196.0,44044.0,328104.0,1205.0,1176.0,5486.0,7160.0,NaN
3,NaN,0.0,0.0,F,N,Red,1.0,Novice,Lava Hot,a,...,3.0,175071.0,274196.0,174188.0,514158.0,1053.0,1056.0,5235.0,3696.0,251.0
4,0.0,NaN,0.0,T,N,Red,3.0,Grandmaster,Cold,h,...,12.0,273832.0,274196.0,253274.0,NaN,1401.0,443.0,7483.0,NaN,898.0


In [172]:
df_test_ce.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,ord_0,ord_1,ord_2,ord_3,...,month,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9
600000,0.0,0.0,0.0,F,Y,Blue,3.0,Novice,Boiling Hot,f,...,9.0,253952.0,254155.0,174188.0,44382.0,1298.0,766.0,8020.0,7136.0,816.0
600001,0.0,0.0,0.0,F,Y,Red,1.0,Novice,Cold,n,...,8.0,175071.0,199206.0,199933.0,328104.0,1542.0,1067.0,8125.0,6783.0,490.0
600002,0.0,0.0,0.0,F,Y,Blue,1.0,Expert,Warm,i,...,6.0,175071.0,254155.0,199933.0,514158.0,1449.0,565.0,7431.0,6221.0,475.0
600003,1.0,0.0,0.0,F,N,Red,1.0,Expert,Hot,m,...,6.0,253952.0,254155.0,253274.0,328104.0,1432.0,936.0,4754.0,5906.0,669.0
600004,0.0,0.0,1.0,F,Y,Red,1.0,Contributor,Lava Hot,o,...,3.0,175071.0,NaN,174188.0,514158.0,920.0,602.0,5754.0,6004.0,523.0


In [176]:
for col in df_train_ce.columns:
    print('column {} ----- {} unique values ---- type: {} '.format(col, df_train_ce[col].nunique(), 
                                                                   df_train_ce[col].dtype.name))

column bin_0 ----- 2 unique values ---- type: float64 
column bin_1 ----- 2 unique values ---- type: float64 
column bin_2 ----- 2 unique values ---- type: float64 
column bin_3 ----- 2 unique values ---- type: object 
column bin_4 ----- 2 unique values ---- type: object 
column nom_0 ----- 3 unique values ---- type: object 
column ord_0 ----- 3 unique values ---- type: float64 
column ord_1 ----- 5 unique values ---- type: object 
column ord_2 ----- 6 unique values ---- type: object 
column ord_3 ----- 15 unique values ---- type: object 
column ord_4 ----- 26 unique values ---- type: object 
column ord_5 ----- 190 unique values ---- type: object 
column day ----- 7 unique values ---- type: float64 
column month ----- 12 unique values ---- type: float64 
column nom_1 ----- 6 unique values ---- type: float64 
column nom_2 ----- 6 unique values ---- type: float64 
column nom_3 ----- 6 unique values ---- type: float64 
column nom_4 ----- 4 unique values ---- type: float64 
column nom_5 --

#### Applying label encoder on the rest of the features:

In [177]:
df_train_ce, df_test_ce, _ = apply_custom_encoder(LabelEncoder, df_train_ce, df_test_ce)

In [178]:
df_train_ce.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,ord_0,ord_1,ord_2,ord_3,...,month,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9
0,0.0,0.0,0.0,0,0,2,3.0,0,3,2,...,3.0,199463.0,274196.0,199933.0,328104.0,653.0,210.0,3037.0,2684.0,323.0
1,1.0,1.0,0.0,0,1,2,3.0,2,5,4,...,7.0,23678.0,254155.0,NaN,514158.0,1146.0,706.0,8161.0,4833.0,879.0
2,0.0,1.0,0.0,0,0,2,3.0,5,2,13,...,9.0,NaN,274196.0,44044.0,328104.0,1205.0,1176.0,5486.0,7160.0,NaN
3,NaN,0.0,0.0,0,0,2,1.0,4,4,0,...,3.0,175071.0,274196.0,174188.0,514158.0,1053.0,1056.0,5235.0,3696.0,251.0
4,0.0,NaN,0.0,1,0,2,3.0,2,1,7,...,12.0,273832.0,274196.0,253274.0,NaN,1401.0,443.0,7483.0,NaN,898.0


In [179]:
len(df_train_ce)

600000

In [180]:
X = df_train_ce

test_size = 0.3
train_size = 0.7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, train_size=train_size, random_state=2021)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

xgb_ce = xgboost.XGBRegressor(n_estimator=20000, max_depth=8, colsample_bytree=0.85,
                           reg_alpha=0.05, reg_lambda=0.05, random_state=2021,learning_rate=0.008, verbosity=1)
xgb_ce.fit(X_train, y_train, eval_set=[(X_val,y_val)], early_stopping_rounds=50, verbose=50 )



[01:56:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:0.49823
[50]	validation_0-rmse:0.43366
[99]	validation_0-rmse:0.40093


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.85, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.008, max_delta_step=0, max_depth=8,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimator=20000, n_estimators=100, n_jobs=8, num_parallel_tree=1,
             random_state=2021, reg_alpha=0.05, reg_lambda=0.05,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=1)

#### hecking model metrics 

In [182]:
pred = xgb_ce.predict(X_test)

In [183]:
metrics.r2_score(y_test, pred)

-0.053285277867557435

In [184]:
metrics.explained_variance_score(y_test, pred)

0.07217765796843356

In [185]:
metrics.max_error(y_test, pred)

0.7642215937376022

In [186]:
metrics.mean_absolute_error(y_test, pred)

0.3783596654350559

In [187]:
metrics.mean_squared_error(y_test, pred)

0.16122801592695257

In [189]:
metrics.mean_poisson_deviance(y_test, pred)

0.6627192297849945

In [188]:
metrics.mean_tweedie_deviance(y_test, pred)

0.16122801592695257

# Applying catboost without any encoding

In [195]:
df_train_cat = pd.read_csv('Train.csv')
y = df_train_cat['target']
df_test_cat = pd.read_csv('Test.csv')
df_train_cat.drop(['id', 'target'], axis=1, inplace=True)
df_test_cat.drop('id', axis=1, inplace=True)

X = df_train_ce
# to avoid: CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=14]=273832.0 : 
#cat_features must be integer or string, real number values and NaN values should be converted to string.
for col in cat_features:
    X[col] = X[col].apply(str)   
    
test_size = 0.3
train_size = 0.7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, train_size=train_size, random_state=2021)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

cat_features = ['bin_3', 'bin_4', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_3',
                'nom_0','nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']



model = CatBoostRegressor(iterations=2000, learning_rate=0.1, depth=10, cat_features=cat_features, thread_count=5)
model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100, early_stopping_rounds=100)

0:	learn: 0.3860891	test: 0.3861059	best: 0.3861059 (0)	total: 1.68s	remaining: 56m 2s
100:	learn: 0.3531081	test: 0.3567355	best: 0.3567355 (100)	total: 3m 17s	remaining: 1h 1m 56s
200:	learn: 0.3482472	test: 0.3561351	best: 0.3561341 (196)	total: 7m 40s	remaining: 1h 8m 44s
300:	learn: 0.3450780	test: 0.3560382	best: 0.3560354 (299)	total: 11m 41s	remaining: 1h 5m 57s
400:	learn: 0.3420804	test: 0.3560642	best: 0.3560242 (347)	total: 15m 56s	remaining: 1h 3m 34s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3560242186
bestIteration = 347

Shrink model to first 348 iterations.


In [199]:
pred = model.predict(X_test)

In [200]:
metrics.r2_score(y_test, pred)

0.1659610983134443

In [201]:
metrics.explained_variance_score(y_test, pred)

0.16599238401317273

In [202]:
metrics.max_error(y_test, pred)

1.073395335724387

In [203]:
metrics.mean_absolute_error(y_test, pred)

0.257680067199237

In [204]:
metrics.mean_squared_error(y_test, pred)

0.1277829971596316

In [205]:
# metrics.mean_poisson_deviance(y_test, pred)

In [ ]:
metrics.mean_tweedie_deviance(y_test, pred)

# Applying Lightgbm on label encoding, onehot encoding for binary values